In [1]:
!pip install Bio Levenshtein -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 55.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.0 MB/s eta 0:00:00


In [2]:
from Bio import SeqIO
import numpy as np
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from scipy.stats import spearmanr, pearsonr
from collections import defaultdict, Counter
from typing import Callable, List, Tuple
import pandas as pd

In [3]:
def evaluate_similarity_method(
    file_path: str,
    fitted_model: object,
    method_name: str = "Custom Method",
    max_sequences: int = 1000
) -> dict:

    sequences = []
    sequence_ids = []

    with open(file_path, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            seq = str(record.seq).strip()
            if len(seq) > 10:
                sequences.append(seq)
                sequence_ids.append(record.id)

    print(f"Loaded {len(sequences)} sequences")

    sequences_subset = sequences[:max_sequences]
    sequence_ids_subset = sequence_ids[:max_sequences]

    if len(sequences_subset) == 0:
        print("No sequences to process after filtering.")
        return {}

    print(f"Using {len(sequences_subset)} sequences for analysis")

    print(f"Calculating similarities using {method_name}...")
    try:
        similarities = fitted_model.predict(sequences_subset[0], sequences_subset)
    except Exception as e:
        print(f"Error during prediction with {method_name}: {e}")
        return {}

    print(f"\nFinding sequences most similar to the first one using {method_name}:")
    print(f"First sequence ID: {sequence_ids_subset[0]}")
    print(f"First sequence length: {len(sequences_subset[0])} amino acids")

    similarity_results = []
    for i in range(len(sequences_subset)):
        similarity_results.append((
            i,
            similarities[i],
            sequence_ids_subset[i],
            sequences_subset[i][:50]
        ))

    similarity_results.sort(key=lambda x: x[1], reverse=True)

    print(f"\nTop 10 most similar sequences to {sequence_ids_subset[0]}:")
    print("-" * 80)
    for i, (idx, sim, seq_id, seq_preview) in enumerate(similarity_results[:10]):
        print(f"{i+1:2d}. Similarity: {sim:.4f} | ID: {seq_id}")
        print(f"    Sequence: {seq_preview}...")
        print()

    print("="*50)
    print(f"METRICS FOR {method_name.upper()}")
    print("="*50)

    ground_truth_ranks = list(range(len(sequences_subset)))

    predicted_similarities = similarities
    predicted_ranks = np.argsort(np.argsort(-predicted_similarities))

    if len(ground_truth_ranks) > 1 and len(np.unique(predicted_ranks)) > 1:
        spearman_corr, spearman_p = spearmanr(ground_truth_ranks, predicted_ranks)
        pearson_corr, pearson_p = pearsonr(ground_truth_ranks, predicted_ranks)
    else:
        spearman_corr = pearson_corr = 0.0
        spearman_p = pearson_p = 1.0

    print(f"Spearman correlation: {spearman_corr:.4f} (p-value: {spearman_p:.4f})")
    print(f"Pearson correlation: {pearson_corr:.4f} (p-value: {pearson_p:.4f})")

    def calculate_ndcg(ground_truth_ranks, predicted_ranks, k=10):
        if len(ground_truth_ranks) == 0:
            return 0.0
        ideal_ranking = list(range(len(ground_truth_ranks)))

        top_k_predicted = np.argsort(predicted_ranks)[:min(k, len(predicted_ranks))]
        top_k_ideal = ideal_ranking[:min(k, len(ideal_ranking))]

        dcg = 0
        for i, idx in enumerate(top_k_predicted):
            rel = 1.0 / (ground_truth_ranks[idx] + 1) if ground_truth_ranks[idx] > 0 else 1.0
            dcg += rel / np.log2(i + 2)

        idcg = 0
        for i in range(len(top_k_ideal)):
            rel = 1.0 / (ideal_ranking[i] + 1) if ideal_ranking[i] > 0 else 1.0
            idcg += rel / np.log2(i + 2)

        return dcg / idcg if idcg > 0 else 0

    ndcg_10 = calculate_ndcg(ground_truth_ranks, predicted_ranks, k=10)
    ndcg_all = calculate_ndcg(ground_truth_ranks, predicted_ranks, k=len(sequences_subset))

    print(f"NDCG@10: {ndcg_10:.4f}")
    print(f"NDCG@all: {ndcg_all:.4f}")

    from scipy.stats import kendalltau
    if len(ground_truth_ranks) > 1:
        tau, tau_p = kendalltau(ground_truth_ranks, predicted_ranks)
    else:
        tau, tau_p = 0.0, 1.0
    print(f"Kendall's Tau: {tau:.4f} (p-value: {tau_p:.4f})")

    print(f"\nSummary for {method_name}:")
    print("-" * 40)
    print(f"Total sequences processed: {len(sequences_subset)}")
    print(f"Ground truth correlation (Spearman): {spearman_corr:.4f}")
    print(f"Ranking quality (NDCG@10): {ndcg_10:.4f}")
    print(f"Kendall's Tau: {tau:.4f}")

    return {
        'method': method_name,
        'sequences_processed': len(sequences_subset),
        'spearman_correlation': spearman_corr,
        'pearson_correlation': pearson_corr,
        'ndcg_10': ndcg_10,
        'ndcg_all': ndcg_all,
        'kendall_tau': tau,
        'spearman_p_value': spearman_p,
        'pearson_p_value': pearson_p,
        'kendall_tau_p_value': tau_p
    }

In [4]:
class LevenshteinSimilarityModel:
    def __init__(self):
        from Levenshtein import distance as levenshtein_distance
        self._levenshtein_distance = levenshtein_distance

    def predict(self, query_sequence: str, sequences: List[str]) -> np.ndarray:
        similarities = []
        for seq in sequences:
            dist = self._levenshtein_distance(query_sequence, seq)
            max_len = max(len(query_sequence), len(seq))
            if max_len > 0:
                similarity = 1 - (dist / max_len)
            else:
                similarity = 1.0
            similarities.append(similarity)
        return np.array(similarities)

    def find_closest_in_fitted_set(self, query_sequence: str, fitted_sequences: List[str]) -> Tuple[str, float]:
        if not fitted_sequences:
            return "", 0.0
        distances = [self._levenshtein_distance(query_sequence, seq) for seq in fitted_sequences]
        min_idx = np.argmin(distances)
        min_dist = distances[min_idx]
        max_len = max(len(query_sequence), len(fitted_sequences[min_idx]))
        similarity = 1 - (min_dist / max_len) if max_len > 0 else 1.0
        return fitted_sequences[min_idx], similarity

In [5]:
class LengthSimilarityModel:
    def __init__(self):
        pass

    def predict(self, query_sequence: str, sequences: List[str]) -> np.ndarray:
        query_seq_len = len(query_sequence)
        similarities = []
        for seq in sequences:
            seq_len = len(seq)
            max_len = max(query_seq_len, seq_len)
            if max_len > 0:
                length_diff = abs(query_seq_len - seq_len)
                normalized_diff = length_diff / max_len
                similarity = 1 - normalized_diff
            else:
                similarity = 1.0
            similarities.append(similarity)
        return np.array(similarities)

    def find_closest_in_fitted_set(self, query_sequence: str, fitted_sequences: List[str]) -> Tuple[str, float]:
        if not fitted_sequences:
            return "", 0.0
        query_len = len(query_sequence)
        lengths = [len(seq) for seq in fitted_sequences]
        diffs = [abs(query_len - l) for l in lengths]
        min_idx = np.argmin(diffs)
        min_diff = diffs[min_idx]
        max_len = max(query_len, lengths[min_idx])
        similarity = 1 - (min_diff / max_len) if max_len > 0 else 1.0
        return fitted_sequences[min_idx], similarity

In [6]:
class ExactMatchSimilarityModel:
    def __init__(self):
        pass

    def predict(self, query_sequence: str, sequences: List[str]) -> np.ndarray:
        similarities = []
        for seq in sequences:
            if seq == query_sequence:
                similarity = 1.0
            else:
                similarity = 0.0
            similarities.append(similarity)
        return np.array(similarities)

    def find_closest_in_fitted_set(self, query_sequence: str, fitted_sequences: List[str]) -> Tuple[str, float]:
        for seq in fitted_sequences:
            if seq == query_sequence:
                return seq, 1.0
        return "", 0.0

In [7]:
class PreFittedGensimFastTextSimilarityModel:
    def __init__(self, k: int = 3, vector_size: int = 100, window: int = 5,
                 min_count: int = 2, epochs: int = 10, workers: int = 4):
        self.k = k
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.epochs = epochs
        self.workers = workers
        self.model = None
        self.fitted = False
        self._fitted_sequences = []

    def _sequence_to_kmers_list(self, sequence):
        if len(sequence) < self.k:
            return [sequence]
        return [sequence[i:i+self.k] for i in range(len(sequence)-self.k+1)]

    def fit(self, sequences: List[str]):
        self._fitted_sequences = sequences[:]
        kmers_sequences = [self._sequence_to_kmers_list(seq) for seq in sequences]

        self.model = FastText(
            sentences=kmers_sequences,
            vector_size=self.vector_size,
            window=self.window,
            min_count=self.min_count,
            workers=self.workers,
            epochs=self.epochs,
            sg=1
        )
        self.fitted = True

    def _get_sequence_vector(self, sequence):
        kmers = self._sequence_to_kmers_list(sequence)
        vectors = []
        for kmer in kmers:
            try:
                vectors.append(self.model.wv[kmer])
            except KeyError:
                try:
                    vec = self.model.wv.word_vec(kmer, use_norm=True)
                    vectors.append(vec)
                except KeyError:
                    continue

        if vectors:
            return np.mean(vectors, axis=0)
        else:
            return np.zeros(self.model.vector_size)

    def predict(self, query_sequence: str, sequences: List[str]) -> np.ndarray:
        if not self.fitted or self.model is None:
            raise ValueError("Model must be fitted before predict.")

        query_vector = self._get_sequence_vector(query_sequence).reshape(1, -1)
        sequence_vectors = np.array([self._get_sequence_vector(seq) for seq in sequences])
        similarities = cosine_similarity(query_vector, sequence_vectors)[0]
        return similarities

    def find_closest_in_fitted_set(self, query_sequence: str, fitted_sequences: List[str] = None) -> Tuple[str, float]:
        if fitted_sequences is None:
            fitted_sequences = self._fitted_sequences
        if not fitted_sequences or not self.fitted:
            return "", 0.0

        query_vector = self._get_sequence_vector(query_sequence).reshape(1, -1)
        fitted_vectors = np.array([self._get_sequence_vector(seq) for seq in fitted_sequences])
        similarities = cosine_similarity(query_vector, fitted_vectors)[0]
        max_idx = np.argmax(similarities)
        return fitted_sequences[max_idx], similarities[max_idx]

In [8]:
class PreFittedKmerTfidfSimilarityModel:
    def __init__(self, k: int = 3, min_freq: int = 2):
        self.k = k
        self.min_freq = min_freq
        self.vectorizer = None
        self.fitted = False
        self._fitted_vocab = None
        self._fitted_sequences = []

    def _generate_kmers(self, sequence):
        return [sequence[i:i+self.k] for i in range(len(sequence) - self.k + 1)]

    def fit(self, sequences: List[str]):
        self._fitted_sequences = sequences[:]
        processed_sequences = []
        for seq in sequences:
            kmers = self._generate_kmers(seq)
            processed_sequences.append(' '.join(kmers))

        self.vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x.split(),
            lowercase=False,
            token_pattern=None,
            min_df=self.min_freq,
            ngram_range=(1, 1)
        )

        _ = self.vectorizer.fit_transform(processed_sequences)
        self.fitted = True
        self._fitted_vocab = set(self.vectorizer.vocabulary_.keys())

    def predict(self, query_sequence: str, sequences: List[str]) -> np.ndarray:
        if not self.fitted or self.vectorizer is None:
            raise ValueError("Model must be fitted before predict.")

        query_kmers = self._generate_kmers(query_sequence)
        processed_query = ' '.join(query_kmers)
        query_vector = self.vectorizer.transform([processed_query])

        processed_sequences = []
        for seq in sequences:
            kmers = self._generate_kmers(seq)
            processed_sequences.append(' '.join(kmers))
        
        sequences_vectors = self.vectorizer.transform(processed_sequences)
        similarities = cosine_similarity(query_vector, sequences_vectors)[0]
        return similarities

    def find_closest_in_fitted_set(self, query_sequence: str, fitted_sequences: List[str] = None) -> Tuple[str, float]:
        if fitted_sequences is None:
            fitted_sequences = self._fitted_sequences
        if not fitted_sequences or not self.fitted:
            return "", 0.0

        query_kmers = self._generate_kmers(query_sequence)
        processed_query = ' '.join(query_kmers)
        query_vector = self.vectorizer.transform([processed_query])

        processed_fitted = []
        for seq in fitted_sequences:
            kmers = self._generate_kmers(seq)
            processed_fitted.append(' '.join(kmers))

        fitted_vectors = self.vectorizer.transform(processed_fitted)
        similarities = cosine_similarity(query_vector, fitted_vectors)[0]
        max_idx = np.argmax(similarities)
        return fitted_sequences[max_idx], similarities[max_idx]

In [9]:
file_path = "/kaggle/input/uniprot-sprot/uniprot_sprot.fasta"

print("Fitting models...")

fit_sequences = []
with open(file_path, "r") as handle:
    for i, record in enumerate(SeqIO.parse(handle, "fasta")):
        if i >= 5000:
            break
        seq = str(record.seq).strip()
        if len(seq) > 10:
            fit_sequences.append(seq)

tfidf_model = PreFittedKmerTfidfSimilarityModel(k=3, min_freq=2)
tfidf_model.fit(fit_sequences)
print("TF-IDF model fitted.")

fasttext_model = PreFittedGensimFastTextSimilarityModel(k=3, vector_size=50, epochs=5)
fasttext_model.fit(fit_sequences)
print("FastText model fitted.")

methods = [
    (LevenshteinSimilarityModel(), "Levenshtein Distance"),
    (LengthSimilarityModel(), "Length-based"),
    (ExactMatchSimilarityModel(), "Exact Match"),
    (tfidf_model, "Pre-fitted TF-IDF (3-mers)"),
    (fasttext_model, "Pre-fitted Gensim FastText (3-mers)"),
]

results = []
for fitted_model, method_name in methods:
    try:
        print(f"\n{'='*20} Evaluating {method_name} {'='*20}")
        max_seqs = 100 if "Levenshtein" in method_name else 1000

        result = evaluate_similarity_method(
            '/kaggle/input/uniprot-sprot/uniprot-choline_esterase_reviewed-yes.fasta',
            fitted_model,
            method_name,
            max_sequences=max_seqs
        )
        if result:
            results.append(result)
        print(f"Completed evaluation for {method_name}\n")
    except Exception as e:
        print(f"Error evaluating {method_name}: {e}")

if results:
    print("\n" + "="*70)
    print("COMPARISON OF ALL METHODS")
    print("="*70)

    comparison_df = pd.DataFrame(results)
    if not comparison_df.empty:
        print_table = comparison_df[[
            'method', 'sequences_processed', 'spearman_correlation',
            'ndcg_10', 'kendall_tau'
        ]].round(4)
        print(print_table.to_string(index=False))
    else:
        print("No results to display.")
else:
    print("No methods were successfully evaluated.")

Fitting models...
TF-IDF model fitted.
FastText model fitted.

==================== Evaluating Levenshtein Distance ====================
Loaded 69 sequences
Using 69 sequences for analysis
Calculating similarities using Levenshtein Distance...

Finding sequences most similar to the first one using Levenshtein Distance:
First sequence ID: sp|P06276|CHLE_HUMAN
First sequence length: 602 amino acids

Top 10 most similar sequences to sp|P06276|CHLE_HUMAN:
--------------------------------------------------------------------------------
 1. Similarity: 1.0000 | ID: sp|P06276|CHLE_HUMAN
    Sequence: MHSKVTIICIRFLFWFLLLCMLIGKSHTEDDIIIATKNGKVRGMNLTVFG...

 2. Similarity: 0.8970 | ID: sp|P32749|CHLE_BOVIN
    Sequence: MQSRSTVIYIRFVLWFLLLWVLFEKSHTEEDIIITTKNGKVRGMHLPVLG...

 3. Similarity: 0.8887 | ID: sp|P21927|CHLE_RABIT
    Sequence: MVTRSSHTEDVIITTKNGRIRGINLPVFGGTVTAFLGIPYAQPPLGRLRF...

 4. Similarity: 0.8704 | ID: sp|O62760|CHLE_FELCA
    Sequence: MQSKGTIISIQFLLRFLLLWVLIGKSHTEEDIIITTKNGKVR